In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_accelerate-training-of-large-embedding-tables-by-lazyadam/nvidia_logo.png" style="width: 90px; float: right;">

# Use multiple optimizers to accelerate training with LazyAdam

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

Merlin Models provide various model APIs for training, as shown in Notebook [Iterating over Deep Learning Models using Merlin Models](https://nvidia-merlin.github.io/models/main/examples/03-Exploring-different-models.html). We can create a model, such as [Two Tower](https://nvidia-merlin.github.io/models/main/models_overview.html?highlight=two%20tower#two-tower), [DLRM](https://nvidia-merlin.github.io/models/main/examples/03-Exploring-different-models.html#dlrm-model) and so on, by simply one line: `model=mm.DLRMModel(schema)`. Some models contain large embedding tables, and training could be slow on such large sparse embeddings. However, this process could be accelerated by using a special optimizer, LazyAdam.

In this example, we utilize LazyAdam for large embedding tables and original Adam for other trainable weights to accelerate the whole training process.


### Learning objectives
- Training a model with multiple optimizers
- Utilizing LazyAdam for training on large embedding tables

In [2]:
import os

import tensorflow as tf

#import merlin
from merlin.datasets.synthetic import generate_data
import merlin.models.tf as ml
from merlin.schema import Schema, Tags

2022-09-20 21:39:49.849995: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-20 21:39:50.972905: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16249 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:2d:00.0, compute capability: 7.0


### Generate Synthetic Dataset
To generate the synthetic dataset for our example, we can use `generate_data()` function.

In [3]:
NUM_ROWS = 1000000
train, valid = generate_data("e-commerce-large", int(NUM_ROWS), set_sizes=(0.7, 0.3))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Create a schema object and remove the target columns.

In [4]:
schema = train.schema.without(['click', 'conversion'])
train.schema = schema
valid.schema = schema

We can print out the schema object.

In [5]:
schema

,name,tags,dtype,is_list,is_ragged,properties.domain.min,properties.domain.max
0,user_categories,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,6086
1,user_shops,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,116741
2,user_brands,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,58015
3,user_intentions,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,33786
4,user_profile,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,98
5,user_group,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,14
6,user_gender,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,3
7,user_age,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,8
8,user_consumption_1,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,4
9,user_consumption_2,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,4


## Building a Two-Tower Model with Merlin Models

In [6]:
item_embeddings = ml.Embeddings(schema.select_by_tag(Tags.ITEM), infer_embedding_sizes=True)
query_embeddings = ml.Embeddings(schema.select_by_tag(Tags.USER), infer_embedding_sizes=True)
model = ml.TwoTowerModel(schema, 
                         item_tower=ml.InputBlockV2(schema.select_by_tag(Tags.ITEM), embeddings=item_embeddings).connect(ml.MLPBlock([64])), 
                         query_tower=ml.InputBlockV2(schema.select_by_tag(Tags.USER), embeddings=query_embeddings).connect(ml.MLPBlock([64])),
                         samplers=[ml.InBatchSampler()],
)

The model initializer would infer the embedding table size from the schema, where the first dimension (`input_dim`) of each embedding table is the same as the cardinalities (categories) of each feature, and the second dimension is specified by the user. By setting `infer_embedding_sizes=True`, the initializer would infer the size based on the cardinalities: 
$$output\_dim=\left \lfloor cardinality^{0.25}\times multiplier \right \rfloor$$
The multiplier is 8 by default.To achieve the best performance with GPU operators, we adjust the embedding dimensions to multiples of 8.

### Apply Multiple Optimizers to the Model

We usually set one optimizer to train a model, but for large embedding tables, at each batch, the weights to be updated could be really sparse, in other words, each time we only update the model based on a small batch of training data, so for a large embedding table (first dimension >>  batch size), at most batch_size rows would be updated. Thus in order to acceleate training on large embedding tables, we want to utilize the Lazy Adam for those large tables. 

Compared with Adam, Lazy Adam is optimized for sparse updates. It only update sparse variables indices for current batch. However it may result in slight difference in experiment results compared with Adam.

#### Split Embedding Tables based on the First Dimension (`input_dim`)
Since we want to apply LazyAdam to the large tables, we have to split all tables into two sets. The result of `split_embeddings_on_size` (i.e. `item_large_tables` and `item_small_tables`) are two lists of embedding tables.


In [7]:
item_large_tables, item_small_tables = ml.split_embeddings_on_size(item_embeddings, threshold=1000)
query_large_tables, query_small_tables = ml.split_embeddings_on_size(query_embeddings, threshold=1000)

We can print out the size of each embedding table:

In [8]:
print("Large embedding tables of query tower:")
for t in query_large_tables:
    print(t.name, "first dimension: ", t.input_dim, "second dimension", t.dim)
print("Small embedding tables of query tower:")
for t in  query_small_tables:
    print(t.name, "first dimension: ", t.input_dim, "second dimension", t.dim)

Large embedding tables of query tower:
user_categories first dimension:  6087 second dimension 24
user_shops first dimension:  116742 second dimension 40
user_brands first dimension:  58016 second dimension 32
user_intentions first dimension:  33787 second dimension 32
user_id first dimension:  294737 second dimension 48
Small embedding tables of query tower:
user_profile first dimension:  99 second dimension 8
user_group first dimension:  15 second dimension 8
user_gender first dimension:  4 second dimension 8
user_age first dimension:  9 second dimension 8
user_consumption_1 first dimension:  5 second dimension 8
user_consumption_2 first dimension:  5 second dimension 8
user_is_occupied first dimension:  4 second dimension 8
user_geography first dimension:  6 second dimension 8


#### Set MultiOptimizer

The `MultiOptimizer` module enables multiple optimizers [(e.g. Adam, SGD, RMSProp, Adagrad)](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) to be used in different layers in parallel. Here we want to apply `LazyAdam` for large embedding tables, and `Adam` for the small embedding tables and all other layers.

In [9]:
optimizer = ml.MultiOptimizer(
                default_optimizer="adam",
                optimizers_and_blocks=[ml.OptimizerBlocks(ml.LazyAdam(), item_large_tables + query_large_tables),
                                       ml.OptimizerBlocks("adam", item_small_tables + query_small_tables)]
                )

Note all other trainable parameters not specified an optimizer would use the `default_optimizer`.

### Train the model and evaluate the training time

In [10]:
model.compile(optimizer=optimizer)
model.fit(train, batch_size=1024, epochs=1)

InvalidArgumentError: Exception encountered when calling layer "user_categories" (type EmbeddingTable).

Can not squeeze dim[1], expected a dimension of 1, got 64 [Op:Squeeze]

Call arguments received by layer "user_categories" (type EmbeddingTable):
  • inputs={'user_categories': 'tf.Tensor(shape=(1024, 64), dtype=float32)'}
  • kwargs={'training': 'False', 'testing': 'False', 'features': {'user_categories': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_shops': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_brands': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_intentions': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_profile': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_group': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_gender': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_age': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_consumption_1': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_consumption_2': 'tf.Tensor(shape=(1024, 1), dtype=int64)', 'user_is

#### Compare training time with Adam optimizer

Now we create same model but this time compile it with Adam optimizer. For this notebook, we use a single Tesla V100-SXM2-32GB-LS. The training result shows that for each step, it costs about 71 ms. And as shown in the experiment above, the training time with Lazy Adam is about 18 ms, it achieves about 4X speed up.

In [ ]:
item_embeddings = ml.Embeddings(schema.select_by_tag(Tags.ITEM), infer_embedding_sizes=True)
query_embeddings = ml.Embeddings(schema.select_by_tag(Tags.USER), infer_embedding_sizes=True)
model = ml.TwoTowerModel(schema, 
                         item_tower=ml.InputBlockV2(schema.select_by_tag(Tags.ITEM), embeddings=item_embeddings).connect(ml.MLPBlock([64])), 
                         query_tower=ml.InputBlockV2(schema.select_by_tag(Tags.USER), embeddings=query_embeddings).connect(ml.MLPBlock([64])),
                         samplers=[ml.InBatchSampler()],
)
model.compile(optimizer="adam")
model.fit(train, batch_size=1024, epochs=1)

That's it! In this example, you learned how to use multiple optimizers for training a Two-Tower model, where one of the optimizers is `LazyAdam` which is variant of the Adam optimizer that handles sparse updates more.